In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# https://data.giss.nasa.gov/gistemp/graphs_v4/

temp_df = pd.read_csv('temp_anomaly.csv')
temp_df = temp_df.rename(columns={'Year':'year',
                          'No_Smoothing':'raw_temp',
                          'Lowess(30)':'smoothed_temp'})
temp_df.head()

,year,raw_temp,smoothed_temp
0,1880,-0.16,-0.20
1,1881,-0.08,-0.20
2,1882,-0.10,-0.21
3,1883,-0.16,-0.21
4,1884,-0.28,-0.21


In [ ]:
# https://datahub.io/core/sea-level-rise#data

sealevel_df = pd.read_csv('csiro_recons_gmsl_yr_2015_csv.csv')
sealevel_df = sealevel_df.rename(columns={'Time':'year'})
# Get only the year from the 'year' column
sealevel_df['year'] = pd.DatetimeIndex(sealevel_df['year']).year

sealevel_df.head()

,year,GMSL,GMSL uncertainty
0,1880,-158.7,24.2
1,1881,-153.1,24.2
2,1882,-169.9,23.0
3,1883,-164.6,22.8
4,1884,-143.7,22.2


In [ ]:
# https://www.co2.earth/historical-co2-datasets

co2_df = pd.read_csv('mole_fraction_of_carbon_dioxide_in_air_input4MIPs_GHGConcentrations_CMIP_UoM-CMIP-1-1-0_gr3-GMNHSH_0000-2014.csv')
co2_df = co2_df.rename(columns={'data_mean_global':'mean_global_co2'})
co2_df.head()

,year,mean_global_co2,data_mean_nh,data_mean_sh
0,0,277.454,277.454,277.454
1,1,277.137,277.137,277.137
2,2,277.160,277.160,277.160
3,3,277.158,277.158,277.158
4,4,277.157,277.157,277.157


In [ ]:
co2_df = co2_df.loc[co2_df['year'] >= 1880]

In [ ]:
merged_df = temp_df.merge(sealevel_df)
merged_df = merged_df.merge(co2_df)
merged_df = merged_df.drop(columns=['GMSL uncertainty', 'data_mean_nh', 'data_mean_sh'])
merged_df.head()

,year,raw_temp,smoothed_temp,GMSL,mean_global_co2
0,1880,-0.16,-0.20,-158.7,289.469999
1,1881,-0.08,-0.20,-153.1,289.736999
2,1882,-0.10,-0.21,-169.9,290.018999
3,1883,-0.16,-0.21,-164.6,290.262999
4,1884,-0.28,-0.21,-143.7,290.511999


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=merged_df['year'],
    y=merged_df['raw_temp'],
    name="Raw temperature data",
    marker=dict(color="#d62728")
))


fig.add_trace(go.Scatter(
    x=merged_df['year'],
    y=merged_df['smoothed_temp'],
    name="Smoothed Temperature Trend",
    marker=dict(color="#d62728")
))

fig.add_trace(go.Scatter(
    x=merged_df['year'],
    y=merged_df['GMSL'],
    name="GMSL (Global Mean Sea Level)",
    yaxis="y2",
    marker=dict(color="royalblue")
))

fig.add_trace(go.Scatter(
    x=merged_df['year'],
    y=merged_df['mean_global_co2'],
    name="Mole fraction of CO<sub>2</sub> in air (global)",
    yaxis="y3",
    marker=dict(color='green')
))


# Create axis objects
fig.update_layout(
    xaxis=dict(
        domain=[0.1, 1]
    ),
    yaxis=dict(
        title="Global Temperature Change",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        )
    ),
    yaxis2=dict(
        title="GMSL (Global Mean Sea Level)",
        titlefont=dict(
            color="royalblue"
        ),
        tickfont=dict(
            color="royalblue"
        ),
        anchor="free",
        overlaying="y",
    ),
    yaxis3=dict(
        title="Atmospheric CO<sub>2</sub>",
        titlefont=dict(
            color='green'
        ),
        tickfont=dict(
            color='green'
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
)

# Update layout properties
fig.update_layout(
    title_text="Global temperature change, Global Mean Sea Level (GMSL) and CO<sub>2</sub> change (1880-2013)",
    width=1400,
)

fig.show()